# EECS 3401 Project 

## Filght Status Prediction


#### Can we predict if the flight is diverted?

In [1]:
import zipfile
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Specify the path to your compressed folder and the CSV file inside
compressed_folder_path = 'Combined_Flights_2022.zip'
csv_file_name = 'Combined_Flights_2022.csv'

# Extract the contents of the ZIP file
with zipfile.ZipFile(compressed_folder_path, 'r') as zip_ref:
    zip_ref.extractall('path/to/extracted/folder')

# Create the full path to the extracted CSV file
csv_file_path = f'path/to/extracted/folder/{csv_file_name}'

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path)
#df = pd.read_csv('Combined_Flights_2022.csv')
df.head(10)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
6,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,LCH,False,False,2139,2136.0,0.0,-3.0,...,2147.0,2213.0,5.0,2231,-13.0,0.0,-1.0,2200-2259,1,0
7,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",TYS,IAH,False,False,1129,1117.0,0.0,-12.0,...,1139.0,1255.0,16.0,1306,5.0,0.0,0.0,1300-1359,4,0
8,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,AEX,False,False,1424,1414.0,0.0,-10.0,...,1430.0,1507.0,6.0,1524,-11.0,0.0,-1.0,1500-1559,1,0
9,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,MOB,False,False,954,947.0,0.0,-7.0,...,1004.0,1104.0,6.0,1121,-11.0,0.0,-1.0,1100-1159,2,0


In [3]:
columns = df.columns
len(columns)

61

In [4]:
columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [5]:
len(df.index)

4078318

In [6]:
origin_airtport_city = df['Origin'].unique()
origin_airtport_city

array(['GJT', 'HRL', 'DRO', 'IAH', 'DEN', 'TYS', 'MOB', 'LNK', 'BHM',
       'IAD', 'ECP', 'AEX', 'LFT', 'BTR', 'ABQ', 'BUF', 'ALB', 'HSV',
       'TUL', 'CPR', 'CRP', 'EWR', 'DCA', 'STL', 'CVG', 'RIC', 'PVD',
       'CLT', 'GSO', 'ERI', 'ORD', 'LEX', 'ABE', 'CAK', 'AVL', 'DAY',
       'CHO', 'ROA', 'PWM', 'LIT', 'MKE', 'BTV', 'OMA', 'GRB', 'MLI',
       'SDF', 'FNT', 'CHS', 'PIT', 'TVC', 'ATW', 'DLH', 'ROC', 'SPI',
       'HPN', 'MBS', 'CID', 'MYR', 'CAE', 'SGF', 'AVP', 'GRR', 'DSM',
       'CLE', 'PIA', 'MSN', 'MDT', 'SBN', 'MEM', 'ICT', 'SHV', 'XNA',
       'DIK', 'GPT', 'PNS', 'LRD', 'LCH', 'MFE', 'FLG', 'BRO', 'SAF',
       'PQI', 'ORF', 'LGA', 'SYR', 'RDU', 'ILM', 'IND', 'PHL', 'CMH',
       'SCE', 'ITH', 'JAX', 'OKC', 'JAN', 'GSP', 'TXK', 'ELP', 'MAF',
       'AUS', 'IDA', 'BNA', 'SAV', 'AMA', 'ATL', 'BDL', 'BLI', 'BOI',
       'BOS', 'BUR', 'BWI', 'BZN', 'COS', 'DAL', 'DTW', 'EUG', 'FAT',
       'FLL', 'GEG', 'HDN', 'HNL', 'HOU', 'ISP', 'ITO', 'KOA', 'LAS',
       'LAX', 'LBB',

In [7]:
busiest_airports = df['Origin'].value_counts().nlargest(10)
busiest_airports

ATL    183697
ORD    172169
DFW    161218
DEN    158092
CLT    126088
LAX    111935
LGA    100260
SEA     98884
LAS     97674
PHX     96095
Name: Origin, dtype: int64

In [8]:
specific_origin_airport = 'ATL'  # Replace with the airport code you're interested in
df_ATL_filtered = df[df['Origin'] == specific_origin_airport]
df_ATL_filtered.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
1855,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,820,832.0,12.0,12.0,...,845.0,1000.0,4.0,1000,4.0,0.0,0.0,1000-1059,4,0
1856,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,1440,1505.0,25.0,25.0,...,1516.0,1625.0,7.0,1615,17.0,1.0,1.0,1600-1659,4,0
1857,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,1955,2105.0,70.0,70.0,...,2123.0,2232.0,5.0,2135,62.0,1.0,4.0,2100-2159,4,0
1858,2022-04-01,Southwest Airlines Co.,ATL,BNA,False,False,1510,1547.0,37.0,37.0,...,1601.0,1539.0,8.0,1510,37.0,1.0,2.0,1500-1559,1,0
1859,2022-04-01,Southwest Airlines Co.,ATL,BNA,False,False,820,819.0,0.0,-1.0,...,833.0,813.0,5.0,835,-17.0,0.0,-2.0,0800-0859,1,0


## Dropping Unwanted columns

These columns represent unique IDs and code which don't provide much value
in the training the model and are unrealated to the data analysis.

In [9]:
# List of columns to drop (IDs)
id_columns_to_drop = ['DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'OriginAirportID',
                      'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID',
                      'DestCityMarketID', 'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline',
                      'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number',
                      'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
                   'OriginCityName', 'OriginState', 'DestCityName', 'DestState',
                      'OriginStateName','OriginStateFips','OriginWac'
                  ,'DestStateName','DestStateFips','DestWac','Operating_Airline']

# Drop ID columns from the DataFrame
df_ATL_filtered = df_ATL_filtered.drop(columns=id_columns_to_drop)

# Print data types of the remaining columns
print(df_ATL_filtered.dtypes)

FlightDate               object
Airline                  object
Origin                   object
Dest                     object
Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
DepDelay                float64
ArrTime                 float64
ArrDelayMinutes         float64
AirTime                 float64
CRSElapsedTime          float64
ActualElapsedTime       float64
Distance                float64
Year                      int64
Quarter                   int64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrDelay                float64
ArrDel15                float64
ArrivalD

## 4. Prepare the data for Machine Learning algorithms.

### - Exploring the columns.

In [10]:
columns = df_ATL_filtered.columns
len(columns)

35

In [11]:
len(df_ATL_filtered.index)

183697

In [12]:
cancel_count = df_ATL_filtered['Cancelled'].value_counts()
cancel_count

False    180265
True       3432
Name: Cancelled, dtype: int64

In [13]:
airlines = df_ATL_filtered['Airline'].unique()
len(airlines)

14

In [14]:
dest = df_ATL_filtered['Dest'].unique()
len(dest)

154

In [15]:
df_ATL_filtered[df_ATL_filtered['Diverted']==True]

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
1935,2022-04-01,Southwest Airlines Co.,ATL,SRQ,False,True,1015,1014.0,0.0,-1.0,...,1031.0,1432.0,4.0,1155,NaN,NaN,NaN,1100-1159,2,1
5391,2022-04-02,Southwest Airlines Co.,ATL,JAN,False,True,1110,1404.0,174.0,174.0,...,1415.0,NaN,NaN,1130,NaN,NaN,NaN,1100-1159,2,1
56528,2022-04-30,Southwest Airlines Co.,ATL,MDW,False,True,1600,1601.0,1.0,1.0,...,1613.0,2140.0,3.0,1650,NaN,NaN,NaN,1600-1659,3,1
56547,2022-04-30,Southwest Airlines Co.,ATL,TPA,False,True,1610,1611.0,1.0,1.0,...,1629.0,2019.0,17.0,1735,NaN,NaN,NaN,1700-1759,2,1
61341,2022-04-06,Southwest Airlines Co.,ATL,FLL,False,True,1645,1710.0,25.0,25.0,...,1755.0,2148.0,3.0,1840,NaN,NaN,NaN,1800-1859,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058990,2022-03-24,Republic Airlines,ATL,DCA,False,True,2109,2105.0,0.0,-4.0,...,2122.0,32.0,3.0,2305,NaN,NaN,NaN,2300-2359,3,1
4069796,2022-03-11,Republic Airlines,ATL,HHH,False,True,1515,1720.0,125.0,125.0,...,1742.0,2024.0,8.0,1625,NaN,NaN,NaN,1600-1659,1,1
4069799,2022-03-12,Republic Airlines,ATL,HHH,False,True,1111,1315.0,124.0,124.0,...,1334.0,NaN,NaN,1221,NaN,NaN,NaN,1200-1259,1,1
4069890,2022-03-19,Republic Airlines,ATL,LEX,False,True,1925,1012.0,887.0,887.0,...,1034.0,1130.0,6.0,2041,NaN,NaN,NaN,2000-2059,2,1


<strong>Note :</strong>

By selecting only true values of Diverted column we see that there are some columns that contain nan. It makes sense because arrival details related columns will be empty because flight never reached actual destination. We have to drop these columns because filling this will not provide any value to the model training instead it will create underfitted outcome. So, we will drop such columns. The list of columns dropped is following.
['ArrDelay', 'ArrDel15', 'ArrivalDelayGroups','ArrDelayMinutes','ActualElapsedTime','AirTime'].

In [16]:
# Drop the columns from the DataFrame
df_ATL_filtered = df_ATL_filtered.drop(columns=['ArrDelay', 'ArrDel15', 'ArrivalDelayGroups','ArrDelayMinutes','ActualElapsedTime','AirTime'])
df_ATL_filtered.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTimeBlk,DistanceGroup,DivAirportLandings
1855,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,820,832.0,12.0,12.0,...,0.0,0800-0859,13.0,845.0,1000.0,4.0,1000,1000-1059,4,0
1856,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,1440,1505.0,25.0,25.0,...,1.0,1400-1459,11.0,1516.0,1625.0,7.0,1615,1600-1659,4,0
1857,2022-04-01,Southwest Airlines Co.,ATL,AUS,False,False,1955,2105.0,70.0,70.0,...,4.0,1900-1959,18.0,2123.0,2232.0,5.0,2135,2100-2159,4,0
1858,2022-04-01,Southwest Airlines Co.,ATL,BNA,False,False,1510,1547.0,37.0,37.0,...,2.0,1500-1559,14.0,1601.0,1539.0,8.0,1510,1500-1559,1,0
1859,2022-04-01,Southwest Airlines Co.,ATL,BNA,False,False,820,819.0,0.0,-1.0,...,-1.0,0800-0859,14.0,833.0,813.0,5.0,835,0800-0859,1,0


In [19]:
df_ATL_filtered[['DepDelay','DepDelayMinutes','ArrTimeBlk','DepTimeBlk','FlightDate','Origin','Dest','DepartureDelayGroups']]

,DepDelay,DepDelayMinutes,ArrTimeBlk,DepTimeBlk,FlightDate,Origin,Dest,DepartureDelayGroups
1855,12.0,12.0,1000-1059,0800-0859,2022-04-01,ATL,AUS,0.0
1856,25.0,25.0,1600-1659,1400-1459,2022-04-01,ATL,AUS,1.0
1857,70.0,70.0,2100-2159,1900-1959,2022-04-01,ATL,AUS,4.0
1858,37.0,37.0,1500-1559,1500-1559,2022-04-01,ATL,BNA,2.0
1859,-1.0,0.0,0800-0859,0800-0859,2022-04-01,ATL,BNA,-1.0
...,...,...,...,...,...,...,...,...
4077926,11.0,11.0,2100-2159,1900-1959,2022-03-31,ATL,IAD,0.0
4077960,35.0,35.0,2200-2259,1900-1959,2022-03-31,ATL,EWR,2.0
4078230,29.0,29.0,2200-2259,1900-1959,2022-03-24,ATL,EWR,1.0
4078259,149.0,149.0,1900-1959,1700-1759,2022-03-13,ATL,EWR,9.0


We can observe follwing - 
1. There are 14 unique airlines taking off from the Atlante airport. This information is required in model training.
2. There are 154 destination where the origin was 'ATL'.
3. ArrTimeBlk and DepTimeBlk columns are represent hourly intervals so we will take in consideration the start of the hour
for example - '1000-1059' we will on consider '10' hours.
4. The columns representing _Delay and _DelayMinutes represent same things but have slight differnce in handling early departures and early arrivals
for our model we to check by using both columns separtly and compare which contributes to more accurate models.
5. Orgin column will be dropped cause its redundant for us.
6. We will also drop the year and quarter columns, the  main reason for that is we are taking data from year 2022 thats why its totally 
useless for us.
7. We need to convert the columns reperesenting time in 'hhmm' format using sine and cosine function to capture their cyclical nature,
if we don't do so then the ml algorithms will plot time as continous form which is not the case


### - Dropping the uneccesary columns

In [21]:
# dropping columns that are not useful for model training.
# Initially we will drop the ArrDelay and DepDelay for the first run
list_cols_drop_ml = ['FlightDate','Year','Airline','Origin','Dest', 'DepDelay','Quarter']
df_ATL_filtered = df_ATL_filtered.drop(columns=list_cols_drop_ml)
print(df_ATL_filtered.dtypes)



Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
CRSElapsedTime          float64
Distance                float64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
dtype: object


In [22]:
columns = df_ATL_filtered.columns
print('Total columns - ' + str(len(columns)))

Total columns - 22


### - Extracting hour from the ArrTimeBlk and DepTimeBlk time intervals

In [23]:
# 'ArrTimeBlk' and 'DepTimeBlk' are the columns with time block intervals
df_ATL_filtered['ArrTimeBlk_numeric'] = df_ATL_filtered['ArrTimeBlk'].str.split('-').str[0].astype(int)

# Repeat the process for 'DepTimeBlk' 
df_ATL_filtered['DepTimeBlk_numeric'] = df_ATL_filtered['DepTimeBlk'].str.split('-').str[0].astype(int)

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes)

Cancelled                  bool
Diverted                   bool
CRSDepTime                int64
DepTime                 float64
DepDelayMinutes         float64
ArrTime                 float64
CRSElapsedTime          float64
Distance                float64
Month                     int64
DayofMonth                int64
DayOfWeek                 int64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrTimeBlk               object
DistanceGroup             int64
DivAirportLandings        int64
ArrTimeBlk_numeric        int32
DepTimeBlk_numeric        int32
dtype: object


In [24]:
# dropping the 'ArrTimeBlk' and DepTimeBlk
df_ATL_filtered = df_ATL_filtered.drop(columns=['DepTimeBlk', 'ArrTimeBlk'])
df_ATL_filtered.head(5)

,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,ArrTime,CRSElapsedTime,Distance,Month,DayofMonth,...,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,DistanceGroup,DivAirportLandings,ArrTimeBlk_numeric,DepTimeBlk_numeric
1855,False,False,820,832.0,12.0,1004.0,160.0,813.0,4,1,...,0.0,13.0,845.0,1000.0,4.0,1000,4,0,1000,800
1856,False,False,1440,1505.0,25.0,1632.0,155.0,813.0,4,1,...,1.0,11.0,1516.0,1625.0,7.0,1615,4,0,1600,1400
1857,False,False,1955,2105.0,70.0,2237.0,160.0,813.0,4,1,...,4.0,18.0,2123.0,2232.0,5.0,2135,4,0,2100,1900
1858,False,False,1510,1547.0,37.0,1547.0,60.0,214.0,4,1,...,2.0,14.0,1601.0,1539.0,8.0,1510,1,0,1500,1500
1859,False,False,820,819.0,0.0,818.0,75.0,214.0,4,1,...,-1.0,14.0,833.0,813.0,5.0,835,1,0,800,800


### - Handling columns with value in 'hhmm' format

In [25]:
# Columns that are in the "hhmm" format
df_ATL_filtered[['CRSDepTime','DepTime','CRSArrTime','ArrTime','WheelsOff','WheelsOn','ArrTimeBlk_numeric','DepTimeBlk_numeric']]


,CRSDepTime,DepTime,CRSArrTime,ArrTime,WheelsOff,WheelsOn,ArrTimeBlk_numeric,DepTimeBlk_numeric
1855,820,832.0,1000,1004.0,845.0,1000.0,1000,800
1856,1440,1505.0,1615,1632.0,1516.0,1625.0,1600,1400
1857,1955,2105.0,2135,2237.0,2123.0,2232.0,2100,1900
1858,1510,1547.0,1510,1547.0,1601.0,1539.0,1500,1500
1859,820,819.0,835,818.0,833.0,813.0,800,800
...,...,...,...,...,...,...,...,...
4077926,1929,1940.0,2120,2116.0,1957.0,2108.0,2100,1900
4077960,1952,2027.0,2220,2258.0,2050.0,2221.0,2200,1900
4078230,1947,2016.0,2200,42.0,2105.0,23.0,2200,1900
4078259,1704,1933.0,1918,2159.0,1955.0,2145.0,1900,1700


In [49]:
time_columns = ['CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime', 'WheelsOff', 'WheelsOn', 'ArrTimeBlk_numeric', 'DepTimeBlk_numeric']

for column in time_columns:
    # Extract hour and minute
    df_ATL_filtered[column + '_hour'] = df_ATL_filtered[column] // 100
    df_ATL_filtered[column + '_minute'] = df_ATL_filtered[column] % 100

    # Apply trigonometric transformations
    df_ATL_filtered[column + '_sin'] = np.sin(2 * np.pi * (df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60))
    df_ATL_filtered[column + '_cos'] = np.cos(2 * np.pi * (df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60))

    # Drop the original columns
    df_ATL_filtered = df_ATL_filtered.drop(columns=[column, column + '_hour', column + '_minute'])

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes)

Cancelled                    bool
Diverted                     bool
DepDelayMinutes           float64
ArrDelayMinutes           float64
AirTime                   float64
CRSElapsedTime            float64
ActualElapsedTime         float64
Distance                  float64
Month                       int64
DayofMonth                  int64
DayOfWeek                   int64
DepDel15                  float64
DepartureDelayGroups      float64
TaxiOut                   float64
TaxiIn                    float64
ArrDel15                  float64
ArrivalDelayGroups        float64
DistanceGroup               int64
DivAirportLandings          int64
CRSDepTime_sin            float64
CRSDepTime_cos            float64
DepTime_sin               float64
DepTime_cos               float64
CRSArrTime_sin            float64
CRSArrTime_cos            float64
ArrTime_sin               float64
ArrTime_cos               float64
WheelsOff_sin             float64
WheelsOff_cos             float64
WheelsOn_sin  

##### Explaination -

1. **Converting 'hhmm' to Minutes:**
   - `df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']`: This part converts the 'hhmm' format to minutes. It takes the hour component, multiplies it by 60 to convert to minutes, and adds the minute component.

2. **Normalizing to [0, 1):**
   - `(df_ATL_filtered[column + '_hour'] * 60 + df_ATL_filtered[column + '_minute']) / (24 * 60)`: This part normalizes the minutes to the range [0, 1) by dividing by the total number of minutes in a day (24 hours * 60 minutes).

3. **Trigonometric Transformations:**
   - `2 * np.pi * ...`: This part scales the normalized time to the range [0, 2π), making it suitable for trigonometric functions.

   - `np.sin(...)`: This calculates the sine of the scaled, normalized time. The sine function produces a wave-like pattern with values between -1 and 1. It captures the cyclical nature of time, where similar times of day will have similar sine values.

   - `np.cos(...)`: Similarly, this calculates the cosine of the scaled, normalized time. The cosine function is another periodic function that is shifted relative to the sine function. It provides an additional perspective on the cyclical pattern.

In summary, the sine and cosine transformations encode the time information into two continuous features that capture both the cyclicality and the phase (shift) within the day. This encoding is often beneficial for machine learning models to capture the temporal patterns in a way that respects the circular nature of time. The resulting features (`column + '_sin'` and `column + '_cos'`) can be used as inputs to your machine learning model for predicting flight cancellations.

### - Converting columns with Boolean values to Binary

In [50]:
'''
df_ATL_filtered['Cancelled_binary'] = df_ATL_filtered['Cancelled'].astype(int)
df_ATL_filtered['Diverted_binary'] = df_ATL_filtered['Diverted'].astype(int)

# Drop the original boolean columns if needed
df_ATL_filtered = df_ATL_filtered.drop(columns=['Cancelled', 'Diverted'])

# Print data types of the updated DataFrame
print(df_ATL_filtered.dtypes) '''

"\ndf_ATL_filtered['Cancelled_binary'] = df_ATL_filtered['Cancelled'].astype(int)\ndf_ATL_filtered['Diverted_binary'] = df_ATL_filtered['Diverted'].astype(int)\n\n# Drop the original boolean columns if needed\ndf_ATL_filtered = df_ATL_filtered.drop(columns=['Cancelled', 'Diverted'])\n\n# Print data types of the updated DataFrame\nprint(df_ATL_filtered.dtypes) "

In [51]:
df_ATL_filtered.head(5)

,Cancelled,Diverted,DepDelayMinutes,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Month,DayofMonth,...,ArrTime_sin,ArrTime_cos,WheelsOff_sin,WheelsOff_cos,WheelsOn_sin,WheelsOn_cos,ArrTimeBlk_numeric_sin,ArrTimeBlk_numeric_cos,DepTimeBlk_numeric_sin,DepTimeBlk_numeric_cos
1855,False,False,12.0,4.0,135.0,160.0,152.0,813.0,4,1,...,0.484810,-0.874620,0.751840,-0.659346,0.500000,-0.866025,0.500000,-0.866025,0.866025,-0.500000
1856,False,False,25.0,17.0,129.0,155.0,147.0,813.0,4,1,...,-0.927184,-0.374607,-0.754710,-0.656059,-0.915311,-0.402747,-0.866025,-0.500000,-0.500000,-0.866025
1857,False,False,70.0,62.0,129.0,160.0,152.0,813.0,4,1,...,-0.354291,0.935135,-0.632705,0.774393,-0.374607,0.927184,-0.707107,0.707107,-0.965926,0.258819
1858,False,False,37.0,37.0,38.0,60.0,60.0,214.0,4,1,...,-0.836286,-0.548293,-0.868199,-0.496217,-0.816642,-0.577145,-0.707107,-0.707107,-0.707107,-0.707107
1859,False,False,0.0,0.0,40.0,75.0,59.0,214.0,4,1,...,0.824126,-0.566406,0.785317,-0.619094,0.836286,-0.548293,0.866025,-0.500000,0.866025,-0.500000


### - Handling missing values in the dataset

In [52]:
# Verify that missing values are filled
missing_values_after_imputation = df_ATL_filtered.isnull().sum()
print("Missing Values :")
print(missing_values_after_imputation)

Missing Values :
Cancelled                    0
Diverted                     0
DepDelayMinutes           3388
ArrDelayMinutes           3845
AirTime                   3845
CRSElapsedTime               0
ActualElapsedTime         3845
Distance                     0
Month                        0
DayofMonth                   0
DayOfWeek                    0
DepDel15                  3388
DepartureDelayGroups      3388
TaxiOut                   3425
TaxiIn                    3453
ArrDel15                  3845
ArrivalDelayGroups        3845
DistanceGroup                0
DivAirportLandings           0
CRSDepTime_sin               0
CRSDepTime_cos               0
DepTime_sin               3388
DepTime_cos               3388
CRSArrTime_sin               0
CRSArrTime_cos               0
ArrTime_sin               3453
ArrTime_cos               3453
WheelsOff_sin             3425
WheelsOff_cos             3425
WheelsOn_sin              3453
WheelsOn_cos              3453
ArrTimeBlk_numeric_sin

<strong>Important Note</strong>
- For the column 'Cancelled' every row value that is True, the following columns <b>['DepTime' 'DepDelayMinutes' 'DepDelay' 'ArrTime'	'ArrDelayMinutes' 'AirTime' 'CRSElapsedTime' 'TaxiOut'	'WheelsOff'	'WheelsOn'	'TaxiIn' 'DepDel15' 'ArrDelay'	'ArrDel15'	'ArrivalDelayGroups']</b> are 'nan'. This means no flight data is collected if the flight is cancelled that adds up.
- For the column 'Diverted'  every row value that is True, the columns containg information about arrival details is 'nan' for those particular rows.
-  to resolve this we will 